<a href="https://colab.research.google.com/github/uandhafb/postcards-from-my-jungle/blob/main/sound_generation_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prompts:

1. Birds: granular flutes with radio static.
2. Whales: two low cellos blue water.
3. Giraffes: rhythmic,fast, urban sounds.
4. Elephants:surrealistic, heavy, alternating short/long sounds.
5. Polar Bears:heavy contrabass scraping volcanic rock with metal breath.
6. Komodo Dragons: stereo tremors crossing left/right in impossible geometry.
7. Horses:galloping fast-slow typewriters patterns.
8. Anemone: blurried barroque choir.
9. Spider Monkey:fast patterns built from pedestrian crosswalk beeps.
10. Penguins:staccato ice beats with a hiphop of short vowels.
11. Owls: ritualistic urban sounds.
12. Foxes: collage of heartbeats making music.
13. Dragonfly:prepared piano strings brushed with aluminum foil wind.
14. Paper animals: collage forest, glitchy guitar.




In [ ]:
#Stable Audio Open 1.0 generates variable-length (up to 47s) stereo audio at 44.1kHz from text prompts
#set up HF_TOKEN

!pip install torchsde
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
from IPython.display import Audio

# --- Adicionado para carregar o token Hugging Face ---
from google.colab import userdata
import os

# Verifica se HF_TOKEN já está definido no ambiente, caso contrário, tenta carregar dos segredos do Colab.
if "HF_TOKEN" not in os.environ:
    try:
        os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
        print("Hugging Face token carregado dos segredos do Colab.")
    except userdata.SecretNotFoundError:
        print("Hugging Face token não encontrado nos segredos do Colab. Por favor, adicione-o como um segredo chamado 'HF_TOKEN'.")
        # Se o token for essencial e não estiver presente, você pode querer sair ou levantar um erro.
        # Por enquanto, a execução continuará, e a próxima linha (pipe.from_pretrained) falhará com o erro 401.
# --------------------------------------------------------

pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# define the prompts
prompt = "staccato ice beats with a hiphop of short vowels."
negative_prompt = "Low quality."

# set the seed for generator
generator = torch.Generator("cuda")

# run the generation
audio = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=200,
    audio_end_in_s=15.0,
    num_waveforms_per_prompt=3,
    generator=generator,
).audios

output = audio[0].T.float().cpu().numpy()
sf.write("output.wav", output, pipe.vae.sampling_rate)

# Play the audio in Colab
Audio("output.wav")